# Basic Neural Network with PyTorch — Iris Dataset

1. Load and inspect data  
2. Train / validation / test split  
3. Data preprocessing (scaling)  
4. Build a basic neural network (MLP)  
5. Training loop  
6. Testing and evaluation  

The Iris dataset is fully numeric and clean, so preprocessing focuses on **scaling and splitting**.

In [1]:
import numpy as np
import pandas as pd

# sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

ModuleNotFoundError: No module named 'pandas'

## 1) Load and Inspect the Iris Dataset

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

feature_names = iris.feature_names
target_names = iris.target_names

df = pd.DataFrame(X, columns=feature_names)
df["label"] = y

df.head(), df["label"].value_counts()

## 2) Train / Validation / Test Split

We split first to avoid data leakage.

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print("Train size:", len(X_train))
print("Val size:  ", len(X_val))
print("Test size: ", len(X_test))

## 3) Data Preprocessing: Feature Scaling

Neural networks are sensitive to feature scales.
We **fit the scaler on training data only**, then apply it to validation and test sets.

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

X_train_scaled.mean(axis=0), X_train_scaled.std(axis=0)

## 4) Convert to PyTorch Tensors and DataLoaders

In [ ]:
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
X_val_t   = torch.tensor(X_val_scaled,   dtype=torch.float32)
X_test_t  = torch.tensor(X_test_scaled,  dtype=torch.float32)

y_train_t = torch.tensor(y_train, dtype=torch.long)
y_val_t   = torch.tensor(y_val,   dtype=torch.long)
y_test_t  = torch.tensor(y_test,  dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t),
                          batch_size=16, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val_t, y_val_t),
                          batch_size=32, shuffle=False)
test_loader  = DataLoader(TensorDataset(X_test_t, y_test_t),
                          batch_size=32, shuffle=False)

next(iter(train_loader))

## 5) Define a Basic Neural Network (MLP)

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.net(x)

model = MLP(input_dim=4, hidden_dim=32, num_classes=3)
model

## 6) Training Setup

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

device

## 7) Training Loop (with Validation Accuracy)

In [ ]:
@torch.no_grad()
def evaluate_accuracy(loader):
    model.eval()
    correct, total = 0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        pred = logits.argmax(dim=1)
        correct += (pred == yb).sum().item()
        total += yb.size(0)
    return correct / total

def train_one_epoch(loader):
    model.train()
    total_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

n_epochs = 100
for epoch in range(1, n_epochs + 1):
    loss = train_one_epoch(train_loader)
    val_acc = evaluate_accuracy(val_loader)
    if epoch % 20 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val Acc: {val_acc:.3f}")

## 8) Final Test Evaluation

In [ ]:
@torch.no_grad()
def predict(loader):
    model.eval()
    y_true, y_pred = [], []
    for xb, yb in loader:
        xb = xb.to(device)
        logits = model(xb)
        pred = logits.argmax(dim=1)
        y_true.append(yb.numpy())
        y_pred.append(pred.cpu().numpy())
    return np.concatenate(y_true), np.concatenate(y_pred)

y_true, y_pred = predict(test_loader)

print("Test accuracy:", accuracy_score(y_true, y_pred))
print("Classification report:", classification_report(y_true, y_pred, target_names=target_names))
print("Confusion matrix:", confusion_matrix(y_true, y_pred))